In [137]:
import json
import base64
import requests
import datetime
import uuid
from pprint import pprint
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

    

In [192]:
alice_verifications = [
    {
         "id":"did:example:alice#key-1",
         "type":"JsonWebKey2020",
         "controller":"did:example:alice",
         "publicKeyJwk":{
            "kty":"OKP",
            "crv":"Ed25519",
            "x":"G-boxFB6vOZBu-wXkm-9Lh79I8nf9Z50cILaOgKKGww"
         }
      },
    {
         "id":"did:example:alice#key-x25519-1",
         "type":"JsonWebKey2020",
         "controller":"did:example:alice",
         "publicKeyJwk":{
            "kty":"OKP",
            "crv":"X25519",
            "x":"avH0O2Y4tqLAq8y9zpianr8ajii5m4F_mICrzNlatXs"
         }
    }
]
secrets_resolver = SecretsResolverDemo()
alice_secrets = [
  {
    "kid":"did:example:alice#key-1",
    "id":"did:example:alice#key-1",
    "kty":"OKP",
    "d":"pFRUKkyzx4kHdJtFSnlPA9WzqkDT1HWV0xZ5OYZd2SY",
    "crv":"Ed25519",
    "x":"G-boxFB6vOZBu-wXkm-9Lh79I8nf9Z50cILaOgKKGww"
  }
]
for secret in alice_secrets:
    await secrets_resolver.add_key(jwk_to_secret(secret))
    
    
bob_verifications = [
     {
        "id":"did:example:bob#key-x25519-1",
        "type":"JsonWebKey2020",
        "controller":"did:example:bob",
        "publicKeyJwk":{
           "kty":"OKP",
           "crv":"X25519",
           "x":"GDTrI66K0pFfO54tlCSvfjjNapIs44dzpneBgyx0S3E"
        }
     }
  ]
bob_secrets = [
  {
    "kid":"did:example:bob#key-x25519-1",
    "id":"did:example:bob#key-x25519-1",
    "kty":"OKP",
    "d":"b9NnuOCB0hm7YGNvaE9DMhwH_wjZA1-gWD6dA0JWdL0",
    "crv":"X25519",
    "x":"GDTrI66K0pFfO54tlCSvfjjNapIs44dzpneBgyx0S3E"
  }
]
for secret in bob_secrets:
    await secrets_resolver.add_key(jwk_to_secret(secret))

In [193]:
class FakeResolver(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        if 'alice' in did:
            return DIDDoc(
                did=did,
                key_agreement_kids = ['did:example:alice#key-1','did:example:alice#key-x25519-1'],
                authentication_kids = ['did:example:alice#key-1'],
                verification_methods = [
                VerificationMethod(
                    id = m['id'],
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m['controller'],
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m['publicKeyJwk'])
                    )
                )
                for m in alice_verifications
            ],
                 didcomm_services = []
            )
        elif 'bob' in did:
            return DIDDoc(
                did=did,
                key_agreement_kids = ['did:example:bob#key-x25519-1'],
                authentication_kids = ['did:example:bob#key-x25519-1'],
                verification_methods = [
                VerificationMethod(
                    id = m['id'],
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m['controller'],
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m['publicKeyJwk'])
                    )
                )
                for m in bob_verifications
            ],
                 didcomm_services = []
            )
        else:
            return false


In [198]:
ping_msg = Message(
    body = { "response_requested": True },
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/trust-ping/2.0/ping",
    frm = 'did:example:alice',
    to = ['did:example:bob']
)


In [221]:
ping_msg_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = FakeResolver()
    ),
    message = ping_msg,
    frm = 'did:example:alice',
    to = 'did:example:bob',
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
ping_msg_packed

PackEncryptedResult(packed_msg='{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbVY0WVcxd2JHVTZZV3hwWTJVamEyVjVMWGd5TlRVeE9TMHgiLCJhcHYiOiJRQkhMSUxyTklCR1E3TnJtb1NnQk40VXZUdUJXN0kzQUJBOV9NT1NDbnl3Iiwic2tpZCI6ImRpZDpleGFtcGxlOmFsaWNlI2tleS14MjU1MTktMSIsImVwayI6eyJjcnYiOiJYMjU1MTkiLCJ4IjoidHVrN2FPVGNCNVo0R0RVMEh2NnU4Ujc1SVB4VGJSUkhYeGhzRWlPOEpYQSIsImt0eSI6Ik9LUCJ9fQ","recipients":[{"header":{"kid":"did:example:bob#key-x25519-1"},"encrypted_key":"mZTVUX_Zir3g8D4ZRvkGapOsS9eJjTv9VYahMYEgfozTcsF8pJ5DjzN-odUOzYydG520r-8qaIFyCAdz7dWM9hOyzawEB7mN"}],"iv":"uaVKf-Vc96aO4CCNG5Saew","ciphertext":"BO1umotiHfg1Pi6FiU9wT4D8AOVQDHYktNtPGPhEC3hftHZdQMj5EwWYzf63JHz0RWXpp2i-hSCDI9u7Z1vuKWBKKTi0JVfwQBPOrVJPi-3cYlx-KhDrlbPmHGYzSgrfdXtPLYbrYf6cvjrAF5sDPdeEujyj4luf_U5xh43tBc_jMdt4hnlTt1fAn8hoyTIo1c2vxp7avL_5E6mAD4F6OxmYJtEBGus6HMEboZliqNqFidON_VAVmpSudzM0O7Uivqks17DwrqJKJvqweonJjbMgbruFjqeRNWkz_yYBBLW8gP9QrY

In [222]:
json.loads(ping_msg_packed.packed_msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbVY0WVcxd2JHVTZZV3hwWTJVamEyVjVMWGd5TlRVeE9TMHgiLCJhcHYiOiJRQkhMSUxyTklCR1E3TnJtb1NnQk40VXZUdUJXN0kzQUJBOV9NT1NDbnl3Iiwic2tpZCI6ImRpZDpleGFtcGxlOmFsaWNlI2tleS14MjU1MTktMSIsImVwayI6eyJjcnYiOiJYMjU1MTkiLCJ4IjoidHVrN2FPVGNCNVo0R0RVMEh2NnU4Ujc1SVB4VGJSUkhYeGhzRWlPOEpYQSIsImt0eSI6Ik9LUCJ9fQ',
 'recipients': [{'header': {'kid': 'did:example:bob#key-x25519-1'},
   'encrypted_key': 'mZTVUX_Zir3g8D4ZRvkGapOsS9eJjTv9VYahMYEgfozTcsF8pJ5DjzN-odUOzYydG520r-8qaIFyCAdz7dWM9hOyzawEB7mN'}],
 'iv': 'uaVKf-Vc96aO4CCNG5Saew',
 'ciphertext': 'BO1umotiHfg1Pi6FiU9wT4D8AOVQDHYktNtPGPhEC3hftHZdQMj5EwWYzf63JHz0RWXpp2i-hSCDI9u7Z1vuKWBKKTi0JVfwQBPOrVJPi-3cYlx-KhDrlbPmHGYzSgrfdXtPLYbrYf6cvjrAF5sDPdeEujyj4luf_U5xh43tBc_jMdt4hnlTt1fAn8hoyTIo1c2vxp7avL_5E6mAD4F6OxmYJtEBGus6HMEboZliqNqFidON_VAVmpSudzM0O7Uivqks17DwrqJKJvqweonJjbMgbruFjqeRNWkz_yYBBLW8gP9QrY8LUapLxPemvUf4K

In [ ]:
await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=FakeResolver()
    ),
    packed_msg= json.loads(ping_msg_packed.packed_msg)
)

## unpack veramo message

In [ ]:
#https://github.com/decentralized-identity/didcomm-messaging/issues/435#issuecomment-1483150078
msg = "{\"protected\":\"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiZW5jIjoiWEMyMFAifQ\",\"iv\":\"LGY0TN6LxddlErFVJSuK6zIS-MeMrMXj\",\"ciphertext\":\"_LbCEaZlGD3oV7ljOksaGimlGtk-GrJTlgvmBVe7_PxdYlBe7M1Z97WWGu4J6aWlTGJoxImnh-E7nbYmkHXnZlayCv3mcjuriomD_2j57GbiluY-lO3yqveHgWrDkh1SOwVfm1pl6OrGrjSsDsEtrYIKS5d2C_lDi3ypVILHA7kdDmVdktHljPHlSEgVirzOqq8PwZIjht4sqRNCyw\",\"tag\":\"KvwWlRKRzfC6O5854C_Z-A\",\"recipients\":[{\"encrypted_key\":\"WpdEE1Qz3TcW_JGNVZBYNzut6hSnF2SBmTpmDMba0YI\",\"header\":{\"alg\":\"ECDH-ES+XC20PKW\",\"iv\":\"gAlWL6c2Ay9CR6VT2JSP2i8wTORTEDVp\",\"tag\":\"9mIh53e9ua2_KtERG-rXWg\",\"epk\":{\"kty\":\"OKP\",\"crv\":\"X25519\",\"x\":\"VR9u58ysZjhmUUv_8y4gs4w1EQwUwHv6QNMsw3UZ2GA\"},\"kid\":\"did:example:bob#key-x25519-1\"}},{\"encrypted_key\":\"vVAEvtQo_EY4CSxCZ0xUIf8lSAmsTNQsIQ-rWILXukM\",\"header\":{\"alg\":\"ECDH-ES+XC20PKW\",\"iv\":\"KUtJdmrT-uNBCXJIAnuSclQup7UM8Bs0\",\"tag\":\"EC_RWpMsegAIWlWs1RPzIw\",\"epk\":{\"kty\":\"OKP\",\"crv\":\"X25519\",\"x\":\"U9CXMzGyk--CGXry3Wn9UJihOMUragqz66eA1WCZkA8\"},\"kid\":\"did:example:bob#key-x25519-2\"}},{\"encrypted_key\":\"9qpBOuvg7hSl3AxtiePLI5adreHLjCHVBjJDaZGw4AU\",\"header\":{\"alg\":\"ECDH-ES+XC20PKW\",\"iv\":\"PlQHRMDEwisiTzMW5nA9fw8tEWlB8iNg\",\"tag\":\"9V4gIvJjgTopUp-SQA9RUA\",\"epk\":{\"kty\":\"OKP\",\"crv\":\"X25519\",\"x\":\"_qYNPgBSldQ60jSfn7gRXJFts2fYyHVy5Ude36TROGc\"},\"kid\":\"did:example:bob#key-x25519-3\"}}]}"
msg

In [223]:
json.loads(msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiZW5jIjoiWEMyMFAifQ',
 'iv': 'LGY0TN6LxddlErFVJSuK6zIS-MeMrMXj',
 'ciphertext': '_LbCEaZlGD3oV7ljOksaGimlGtk-GrJTlgvmBVe7_PxdYlBe7M1Z97WWGu4J6aWlTGJoxImnh-E7nbYmkHXnZlayCv3mcjuriomD_2j57GbiluY-lO3yqveHgWrDkh1SOwVfm1pl6OrGrjSsDsEtrYIKS5d2C_lDi3ypVILHA7kdDmVdktHljPHlSEgVirzOqq8PwZIjht4sqRNCyw',
 'tag': 'KvwWlRKRzfC6O5854C_Z-A',
 'recipients': [{'encrypted_key': 'WpdEE1Qz3TcW_JGNVZBYNzut6hSnF2SBmTpmDMba0YI',
   'header': {'alg': 'ECDH-ES+XC20PKW',
    'iv': 'gAlWL6c2Ay9CR6VT2JSP2i8wTORTEDVp',
    'tag': '9mIh53e9ua2_KtERG-rXWg',
    'epk': {'kty': 'OKP',
     'crv': 'X25519',
     'x': 'VR9u58ysZjhmUUv_8y4gs4w1EQwUwHv6QNMsw3UZ2GA'},
    'kid': 'did:example:bob#key-x25519-1'}},
  {'encrypted_key': 'vVAEvtQo_EY4CSxCZ0xUIf8lSAmsTNQsIQ-rWILXukM',
   'header': {'alg': 'ECDH-ES+XC20PKW',
    'iv': 'KUtJdmrT-uNBCXJIAnuSclQup7UM8Bs0',
    'tag': 'EC_RWpMsegAIWlWs1RPzIw',
    'epk': {'kty': 'OKP',
     'crv': 'X25519',
     

In [235]:
await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=FakeResolver()
    ),
    packed_msg= json.loads(msg)
)

MalformedMessageError: MalformedMessageCode.INVALID_PLAINTEXT

### Create Secrets and DID resolvers

In [6]:
secrets_resolver = SecretsResolverDemo()

class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
             didcomm_services = []
        )
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            print(private_key)
            print(jwk_to_secret(private_key))
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did

### 1. Tony get OOB from Mediator
Didcomm mediator: https://mediator.rootsid.cloud/

In [231]:
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
pprint(received_msg_decoded)

{'body': {'accept': ['didcomm/v2'],
          'goal': 'RequestMediate',
          'goal_code': 'request-mediate',
          'label': 'Mediator'},
 'from': 'did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2sQVYAxaeWhE.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0',
 'id': '6554aef2-4b6e-42e7-a8fa-3e9f515acfaf',
 'type': 'https://didcomm.org/out-of-band/2.0/invitation'}


### 2. Tony creates peer did with mediator

In [232]:
tony_did_for_trust_ping = await create_peer_did(2,)
print("Tony's DID:", tony_did_for_trust_ping)

{'crv': 'X25519', 'x': 'DDNfhQ6m37cumPOXQv1oUNPr8Rwp5nksOlXNBo-3FWc', 'd': 'iGUSnUPDxbQUMmp7RLpOuxvglt-nxv4Q3sqtn9B7NX8', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe.Vz6MksY93ZAAJ9NcLrymU8pPjUoNXwfU4HVGuFP1mqQqhrsp2#6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe'}
Secret(kid='did:peer:2.Ez6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe.Vz6MksY93ZAAJ9NcLrymU8pPjUoNXwfU4HVGuFP1mqQqhrsp2#6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe', type=<VerificationMethodType.JSON_WEB_KEY_2020: 1>, verification_material=VerificationMaterial(format=<VerificationMaterialFormat.JWK: 1>, value='{"crv":"X25519","x":"DDNfhQ6m37cumPOXQv1oUNPr8Rwp5nksOlXNBo-3FWc","d":"iGUSnUPDxbQUMmp7RLpOuxvglt-nxv4Q3sqtn9B7NX8","kty":"OKP","kid":"did:peer:2.Ez6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe.Vz6MksY93ZAAJ9NcLrymU8pPjUoNXwfU4HVGuFP1mqQqhrsp2#6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe"}'))
Tony's DID: did:peer:2.Ez6LScVoVVRCwrx8pubDHpwSAFxprXJqxYKb9m6pVqUgwhqNe.Vz6M

### 3. Tony checks the Mediator Trust ping Endpoint

In [233]:
ping_msg = Message(
    body = { "response_requested": True },
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/trust-ping/2.0/ping",
    frm = tony_did_for_trust_ping,
    to = [received_msg_decoded["from"]]
)

tony_trust_ping_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = ping_msg,
    frm = tony_did_for_trust_ping,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
print(tony_trust_ping_packed)
trust_ping_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
trust_ping_endpoint = trust_ping_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(trust_ping_endpoint, headers=headers, json = json.loads(tony_trust_ping_packed.packed_msg))
trust_ping_response = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(trust_ping_response.message.type)#https://didcomm.org/trust-ping/2.0/ping-response

PackEncryptedResult(packed_msg='{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJOV2IxWldVa04zY25nNGNIVmlSRWh3ZDFOQlJuaHdjbGhLY1hoWlMySTViVFp3Vm5GVlozZG9jVTVsTGxaNk5rMXJjMWs1TTFwQlFVbzVUbU5NY25sdFZUaHdVR3BWYjA1WWQyWlZORWhXUjNWR1VERnRjVkZ4YUhKemNESWpOa3hUWTFadlZsWlNRM2R5ZURod2RXSkVTSEIzVTBGR2VIQnlXRXB4ZUZsTFlqbHRObkJXY1ZWbmQyaHhUbVUiLCJhcHYiOiJlUXhXSUxDSGo2NWlYeVNybkJzRTNjTVBMdjZIcTBVVnlKUGZ5Ykt0N3lRIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNjVm9WVlJDd3J4OHB1YkRIcHdTQUZ4cHJYSnF4WUtiOW02cFZxVWd3aHFOZS5WejZNa3NZOTNaQUFKOU5jTHJ5bVU4cFBqVW9OWHdmVTRIVkd1RlAxbXFRcWhyc3AyIzZMU2NWb1ZWUkN3cng4cHViREhwd1NBRnhwclhKcXhZS2I5bTZwVnFVZ3docU5lIiwiZXBrIjp7ImNydiI6IlgyNTUxOSIsIngiOiJITUQ5SHQzbUtEUlZjT0Y4b2JFS1lQWEs3eUdRdzdjY1BScUNjOXJBcmxBIiwia3R5IjoiT0tQIn19","recipients":[{"header":{"kid":"did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2s

In [234]:
tony_trust_ping_packed

PackEncryptedResult(packed_msg='{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJOV2IxWldVa04zY25nNGNIVmlSRWh3ZDFOQlJuaHdjbGhLY1hoWlMySTViVFp3Vm5GVlozZG9jVTVsTGxaNk5rMXJjMWs1TTFwQlFVbzVUbU5NY25sdFZUaHdVR3BWYjA1WWQyWlZORWhXUjNWR1VERnRjVkZ4YUhKemNESWpOa3hUWTFadlZsWlNRM2R5ZURod2RXSkVTSEIzVTBGR2VIQnlXRXB4ZUZsTFlqbHRObkJXY1ZWbmQyaHhUbVUiLCJhcHYiOiJlUXhXSUxDSGo2NWlYeVNybkJzRTNjTVBMdjZIcTBVVnlKUGZ5Ykt0N3lRIiwic2tpZCI6ImRpZDpwZWVyOjIuRXo2TFNjVm9WVlJDd3J4OHB1YkRIcHdTQUZ4cHJYSnF4WUtiOW02cFZxVWd3aHFOZS5WejZNa3NZOTNaQUFKOU5jTHJ5bVU4cFBqVW9OWHdmVTRIVkd1RlAxbXFRcWhyc3AyIzZMU2NWb1ZWUkN3cng4cHViREhwd1NBRnhwclhKcXhZS2I5bTZwVnFVZ3docU5lIiwiZXBrIjp7ImNydiI6IlgyNTUxOSIsIngiOiJITUQ5SHQzbUtEUlZjT0Y4b2JFS1lQWEs3eUdRdzdjY1BScUNjOXJBcmxBIiwia3R5IjoiT0tQIn19","recipients":[{"header":{"kid":"did:peer:2.Ez6LSms555YhFthn1WV8ciDBpZm86hK9tp83WojJUmxPGk1hZ.Vz6MkmdBjMyB4TS5UbbQw54szm8yvMMf1ftGV2s